## Netflix Analysis
### Phase 2 : Normalization
### Name : Gagan Dhanapune
### gmail : gagandhanapune@gmail.com

In [0]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder , OrdinalEncoder

In [0]:
df = pd.read_csv("/Volumes/workspace/default/netflix/netflix_cleaned_new.csv")

In [0]:
df.head(5)

,show_id,title,director,cast,country,date_added,release_year,rating,description,duration_num,duration_type,rating_encoded,country_encoded,Action & Adventure,Anime Features,Anime Series,British TV Shows,Children & Family Movies,Classic & Cult TV,Classic Movies,Comedies,Crime TV Shows,Cult Movies,Documentaries,Docuseries,Dramas,Faith & Spirituality,Horror Movies,Independent Movies,International Movies,International TV Shows,Kids' TV,Korean TV Shows,LGBTQ Movies,Movies,Music & Musicals,Reality TV,Romantic Movies,Romantic TV Shows,Sci-Fi & Fantasy,Science & Nature TV,Spanish-Language TV Shows,Sports Movies,Stand-Up Comedy,Stand-Up Comedy & Talk Shows,TV Action & Adventure,TV Comedies,TV Dramas,TV Horror,TV Mysteries,TV Sci-Fi & Fantasy,TV Shows,TV Thrillers,Teen TV Shows,Thrillers,is_movie,is_tv_show
0,s1,Dick Johnson Is Dead,Kirsten Johnson,NA,United States,"September 25, 2021",2020,PG-13,"As her father nears the end of his life, filmm...",90.0,min,7.0,604,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,s2,Blood & Water,NA,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,"After crossing paths at a party, a Cape Town t...",2.0,Season,10.0,427,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1
2,s3,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NA,"September 24, 2021",2021,TV-MA,To protect his family from a powerful drug lor...,1.0,Season,10.0,353,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
3,s4,Jailbirds New Orleans,NA,NA,NA,"September 24, 2021",2021,TV-MA,"Feuds, flirtations and toilet talk go down amo...",1.0,Season,10.0,353,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,s5,Kota Factory,NA,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,In a city of coaching centers known to train I...,2.0,Season,10.0,251,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8804 entries, 0 to 8806
Data columns (total 57 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   show_id                       8804 non-null   object 
 1   title                         8804 non-null   object 
 2   director                      8804 non-null   object 
 3   cast                          8804 non-null   object 
 4   country                       8804 non-null   object 
 5   date_added                    8804 non-null   object 
 6   release_year                  8804 non-null   int64  
 7   rating                        8804 non-null   object 
 8   description                   8804 non-null   object 
 9   duration_num                  8804 non-null   float64
 10  duration_type                 8804 non-null   object 
 11  rating_encoded                8804 non-null   float64
 12  country_encoded               8804 non-null   int64  
 13  Act

In [0]:
df.shape

(8804, 57)

In [0]:
df.columns

Index(['show_id', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'description', 'duration_num',
       'duration_type', 'rating_encoded', 'country_encoded',
       'Action & Adventure', 'Anime Features', 'Anime Series',
       'British TV Shows', 'Children & Family Movies', 'Classic & Cult TV',
       'Classic Movies', 'Comedies', 'Crime TV Shows', 'Cult Movies',
       'Documentaries', 'Docuseries', 'Dramas', 'Faith & Spirituality',
       'Horror Movies', 'Independent Movies', 'International Movies',
       'International TV Shows', 'Kids' TV', 'Korean TV Shows', 'LGBTQ Movies',
       'Movies', 'Music & Musicals', 'Reality TV', 'Romantic Movies',
       'Romantic TV Shows', 'Sci-Fi & Fantasy', 'Science & Nature TV',
       'Spanish-Language TV Shows', 'Sports Movies', 'Stand-Up Comedy',
       'Stand-Up Comedy & Talk Shows', 'TV Action & Adventure', 'TV Comedies',
       'TV Dramas', 'TV Horror', 'TV Mysteries', 'TV Sci-Fi & Fantasy',
       '

In [0]:
df.fillna('NA', inplace=True)

In [0]:
df.isnull().sum()

show_id          0
type             0
title            0
director         0
cast             0
country          0
date_added       0
release_year     0
rating           0
listed_in        0
description      0
duration_num     0
duration_type    0
dtype: int64

In [0]:
df['rating'].unique()

array(['PG-13', 'TV-MA', 'PG', 'TV-14', 'TV-PG', 'TV-Y', 'TV-Y7', 'R',
       'TV-G', 'G', 'NC-17', '74 min', '84 min', '66 min', 'NR', 'NA',
       'TV-Y7-FV', 'UR'], dtype=object)

In [0]:
df = df[~df['rating'].isin(['74 min', '84 min', '66 min'])]

In [0]:
df['rating'].unique()

array(['PG-13', 'TV-MA', 'PG', 'TV-14', 'TV-PG', 'TV-Y', 'TV-Y7', 'R',
       'TV-G', 'G', 'NC-17', 'NR', 'NA', 'TV-Y7-FV', 'UR'], dtype=object)

In [0]:
rating_order = [["Not found", "G", "TV-Y", "TV-Y7", "TV-G",
                 "PG", "TV-PG", "PG-13", "TV-14", "R", "TV-MA",
                 "NA", "NR", "NC-17", "TV-Y7-FV", "UR"]]


In [0]:
ordinal_encoder = OrdinalEncoder(
    categories=rating_order,
    handle_unknown="use_encoded_value",
    unknown_value=-1
)
df['rating_encoded'] = ordinal_encoder.fit_transform(df[['rating']])

In [0]:
# Label Encoding for Country
label_encoder = LabelEncoder()
df['country_encoded'] = label_encoder.fit_transform(df['country'].astype(str))

In [0]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,duration_num,duration_type,rating_encoded,country_encoded
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NA,United States,"September 25, 2021",2020,PG-13,Documentaries,"As her father nears the end of his life, filmm...",90.0,min,7.0,604
1,s2,TV Show,Blood & Water,NA,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",2.0,Season,10.0,427
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NA,"September 24, 2021",2021,TV-MA,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,1.0,Season,10.0,353
3,s4,TV Show,Jailbirds New Orleans,NA,NA,NA,"September 24, 2021",2021,TV-MA,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",1.0,Season,10.0,353
4,s5,TV Show,Kota Factory,NA,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,2.0,Season,10.0,251


In [0]:
# df["country"].unique()
# df["country_encoded"].unique()
# df["rating"].unique()
df["rating_encoded"].unique()

array([ 7., 10.,  5.,  8.,  6.,  2.,  3.,  9.,  4.,  1., 13., 12., 11.,
       14., 15.])

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer

In [0]:
# 'listed_in' contains genres as a comma-separated string, e.g., "Drama, Comedy, Action"
# We want a list for each row ["Drama", "Comedy", "Action"]
df['listed_in'] = df['listed_in'].apply(
    lambda x: [i.strip() for i in x.split(',')] if isinstance(x, str) else []
)

In [0]:
mlb = MultiLabelBinarizer()
# MultiLabelBinarizer is used when each row can have multiple labels (genres)
# It will create a new column for each unique genre across all rows with 0/1 values


In [0]:
genre_encoded = pd.DataFrame(
    mlb.fit_transform(df['listed_in']),  # Transform the list of genres into binary 0/1 columns
    columns=mlb.classes_,                # Use the genre names as column headers
    index=df.index                      
)

In [0]:
df = pd.concat([df.drop(columns=['listed_in']), genre_encoded], axis=1)

In [0]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,description,duration_num,duration_type,rating_encoded,country_encoded,Action & Adventure,Anime Features,Anime Series,British TV Shows,Children & Family Movies,Classic & Cult TV,Classic Movies,Comedies,Crime TV Shows,Cult Movies,Documentaries,Docuseries,Dramas,Faith & Spirituality,Horror Movies,Independent Movies,International Movies,International TV Shows,Kids' TV,Korean TV Shows,LGBTQ Movies,Movies,Music & Musicals,Reality TV,Romantic Movies,Romantic TV Shows,Sci-Fi & Fantasy,Science & Nature TV,Spanish-Language TV Shows,Sports Movies,Stand-Up Comedy,Stand-Up Comedy & Talk Shows,TV Action & Adventure,TV Comedies,TV Dramas,TV Horror,TV Mysteries,TV Sci-Fi & Fantasy,TV Shows,TV Thrillers,Teen TV Shows,Thrillers
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NA,United States,"September 25, 2021",2020,PG-13,"As her father nears the end of his life, filmm...",90.0,min,7.0,604,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,s2,TV Show,Blood & Water,NA,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,"After crossing paths at a party, a Cape Town t...",2.0,Season,10.0,427,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NA,"September 24, 2021",2021,TV-MA,To protect his family from a powerful drug lor...,1.0,Season,10.0,353,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,s4,TV Show,Jailbirds New Orleans,NA,NA,NA,"September 24, 2021",2021,TV-MA,"Feuds, flirtations and toilet talk go down amo...",1.0,Season,10.0,353,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,s5,TV Show,Kota Factory,NA,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,In a city of coaching centers known to train I...,2.0,Season,10.0,251,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [0]:
df['is_movie'] = df['type'].apply(lambda x: 1 if x == 'Movie' else 0)
df['is_tv_show'] = df['type'].apply(lambda x: 1 if x == 'TV Show' else 0)

In [0]:
#droping old Categorical Column
columns_to_drop = ['rating', 'country']
df.drop(columns=columns_to_drop, inplace=True)

In [0]:
df.drop(columns=['type'], inplace=True)

In [0]:
df.head(5)

,show_id,title,director,cast,date_added,release_year,description,duration_num,duration_type,rating_encoded,country_encoded,Action & Adventure,Anime Features,Anime Series,British TV Shows,Children & Family Movies,Classic & Cult TV,Classic Movies,Comedies,Crime TV Shows,Cult Movies,Documentaries,Docuseries,Dramas,Faith & Spirituality,Horror Movies,Independent Movies,International Movies,International TV Shows,Kids' TV,Korean TV Shows,LGBTQ Movies,Movies,Music & Musicals,Reality TV,Romantic Movies,Romantic TV Shows,Sci-Fi & Fantasy,Science & Nature TV,Spanish-Language TV Shows,Sports Movies,Stand-Up Comedy,Stand-Up Comedy & Talk Shows,TV Action & Adventure,TV Comedies,TV Dramas,TV Horror,TV Mysteries,TV Sci-Fi & Fantasy,TV Shows,TV Thrillers,Teen TV Shows,Thrillers,is_movie,is_tv_show
0,s1,Dick Johnson Is Dead,Kirsten Johnson,NA,"September 25, 2021",2020,"As her father nears the end of his life, filmm...",90.0,min,7.0,604,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,s2,Blood & Water,NA,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...","September 24, 2021",2021,"After crossing paths at a party, a Cape Town t...",2.0,Season,10.0,427,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1
2,s3,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...","September 24, 2021",2021,To protect his family from a powerful drug lor...,1.0,Season,10.0,353,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
3,s4,Jailbirds New Orleans,NA,NA,"September 24, 2021",2021,"Feuds, flirtations and toilet talk go down amo...",1.0,Season,10.0,353,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,s5,Kota Factory,NA,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...","September 24, 2021",2021,In a city of coaching centers known to train I...,2.0,Season,10.0,251,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1


In [0]:
df.shape

(8804, 55)

In [0]:
df.to_csv("/Volumes/workspace/default/netflix/netflix_normalized_new.csv", index=False)
print("Normalized dataset saved as 'netflix_normalized_new.csv'")

Normalized dataset saved as 'netflix_normalized_new.csv'
